<a href="https://colab.research.google.com/github/Simon-Pu/Temp/blob/master/Hyperparameter_Optimization_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

In [0]:
from torch import *
%matplotlib inline

from torchvision.models import densenet201
from torchvision.models import resnext50_32x4d
from torchvision.datasets import CIFAR10
from torchvision import transforms

from torch.utils.data import DataLoader
from torch.optim import Adam
import torch.nn.functional as F
from torch.nn import Linear
import torch

print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

In [0]:
#model = densenet201(pretrained = True)
model = resnext50_32x4d(pretrained = True)
model.classifier = Linear(1920,10)

# Move model to GPU for faster training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set model to training mode
model.train()

In [0]:
# Change the value of bs to change the batch size
bs = 256

train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


train_ds = CIFAR10(root='./data', train=True, download=True, transform=train_tfms)
test_ds = CIFAR10(root='./data', train=False, download=True, transform=test_tfms)

train_dl = DataLoader(train_ds, batch_size = bs)
test_dl = DataLoader(test_ds , batch_size = bs)

In [0]:

loss_func = F.cross_entropy
  # Define function to check model accuracy
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

# Change the value of lr to change the learning rate
lr = 1e-3
optim = Adam(model.parameters(), lr = lr)

# Change the value of epochs to change the number of epochs
epochs = 5

for epoch in range(epochs):
    
    for xb, yb in train_dl:
        
        # .to(device) moves torch.Tensor objects to the GPU for faster training
        
        preds = model(xb.to(device))
        loss = loss_func(preds, yb.to(device))
        acc = accuracy(preds,yb.to(device))

        loss.backward()
        optim.step()
        optim.zero_grad()
        
    print("Loss: " + str(loss.item()) + "\t \t Accuracy: " + str(100 * acc.item()))

In [0]:
def get_model_accuracy():
  tot_acc = 0
  avg_acc = 0
  
  # Set model to evaluation mode
  model.eval()

  for xbt, ybt in test_dl:

    pred = model(xbt.to(device))
    tot_acc += accuracy(pred,ybt.to(device))

  avg_acc = tot_acc / len(test_dl)
  
  return avg_acc.item()


# Print accuracy of model
print("The average accuracy is: " + str(get_model_accuracy()))

In [0]:
!pip install bayesian-optimization

In [0]:
def obj_func(lr, bs, epochs):
      
      # We need to round off bs and epochs because Gaussian processes cannot deal with discrete variables 
      bs = int(bs)
      epochs = int(epochs)
      
      train_dl = DataLoader(train_ds, batch_size = bs)
      test_dl = DataLoader(test_ds , batch_size = bs)
      
      optim = Adam(model.parameters(), lr = lr)
      
      for epoch in range(epochs):
    
        for xb, yb in train_dl:
        
            # .to(device) moves torch.Tensor objects to the GPU for faster training
        
            preds = model(xb.to(device))
            loss = loss_func(preds, yb.to(device))
            acc = accuracy(preds,yb.to(device))
        
            loss.backward()
            optim.step()
            optim.zero_grad()
        
        print("Loss: " + str(loss.item()) + "\t \t Accuracy: " + str(100 * acc.item()))

      acc = get_model_accuracy()
      
      return acc

In [0]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'lr': (1e-4, 1e-2), 'bs': (64, 512), 'epochs': (1,25)}

optimizer = BayesianOptimization(
    f=obj_func,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=2, n_iter=3,)

print(optimizer.max)

In [0]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {'lr': (1e-4, 1e-2), 'bs': (64, 512), 'epochs': (1,25)}

optimizer = BayesianOptimization(
    f=obj_func,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=2, n_iter=3,)

print(optimizer.max)

In [0]:
!pip install optuna

In [0]:
def objective(trial):
      # We need to round off bs and epochs because Gaussian processes cannot deal with discrete variables 
      #bs = int(bs)
      #epochs = int(epochs)
      
      # Generate the optimizers.
      lr = trial.suggest_uniform("lr", 1e-4, 1e-2)
      bs = trial.suggest_int('bs', 32, 265)
      epochs = trial.suggest_int('epochs', 5, 5)

      print(lr, bs, epochs)
      train_dl = DataLoader(train_ds, batch_size = bs)
      test_dl = DataLoader(test_ds , batch_size = bs)
      
      optim = Adam(model.parameters(), lr = lr)
      
      for epoch in range(epochs):
    
        for xb, yb in train_dl:
        
            # .to(device) moves torch.Tensor objects to the GPU for faster training
        
            preds = model(xb.to(device))
            loss = loss_func(preds, yb.to(device))
            acc = accuracy(preds,yb.to(device))
        
            loss.backward()
            optim.step()
            optim.zero_grad()
        
        print("Loss: " + str(loss.item()) + "\t \t Accuracy: " + str(100 * acc.item()))

      acc = get_model_accuracy()
      
      return acc  

In [0]:
import optuna
   
study = optuna.create_study(direction="maximize")
#study.optimize(objective, n_trials=100)
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
   print("    {}: {}".format(key, value))